## 事件流处理器

In [32]:
from illufly.io import EventBlock
EventBlock("Chunk", "hi")

EventBlock(block_type=<chunk>, content=<hi>)

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [33]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

hi
illufly!


In [34]:
r.events_history.store

{'20241129-70470-0004-4087': {'agents': {'MyRun.4554695216': {}},
  'callings': {'20241129-70470-0004-7364': [{'id': '20241129-70470-0047-9401',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0047-9401", "block_type": "runnable", "content": "MyRun.4554695216", "content_id": "20241129-70470-0048-5561", "created_at": "2024-11-29T16:54:30.790858", "thread_id": null, "calling_id": "20241129-70470-0004-7364", "agent_name": "MyRun.4554695216", "model_name": null}'},
    {'id': '20241129-70470-0049-6304',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0049-6304", "block_type": "text", "content": "hi\\n", "content_id": "20241129-70470-0050-2090", "created_at": "2024-11-29T16:54:30.790924", "thread_id": null, "calling_id": "20241129-70470-0004-7364", "agent_name": "MyRun.4554695216", "model_name": null}'},
    {'id': '20241129-70470-0051-0908',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0051-0908", "block_type": "text", "content

### 推送事件流

In [35]:
for x in r(generator="sync", verbose=False):
    print("SSE >>", x)

SSE >> {'id': '20241129-70470-0057-1868', 'event': 'message', 'data': '{"block_id": "20241129-70470-0057-1868", "block_type": "text", "content": "hi\\n", "content_id": "20241129-70470-0058-1982", "created_at": "2024-11-29T16:54:30.822266", "thread_id": null, "calling_id": "20241129-70470-0005-9651", "agent_name": "MyRun.4554695216", "model_name": null}'}
SSE >> {'id': '20241129-70470-0061-4565', 'event': 'message', 'data': '{"block_id": "20241129-70470-0061-4565", "block_type": "text", "content": "illufly!\\n", "content_id": "20241129-70470-0062-1740", "created_at": "2024-11-29T16:54:30.822651", "thread_id": null, "calling_id": "20241129-70470-0005-9651", "agent_name": "MyRun.4554695216", "model_name": null}'}


In [36]:
async for x in r(generator="async", verbose=False):
    print("SSE >>", x)

SSE >> {'id': '20241129-70470-0067-5472', 'event': 'message', 'data': '{"block_id": "20241129-70470-0067-5472", "block_type": "text", "content": "hi\\n", "content_id": "20241129-70470-0068-4138", "created_at": "2024-11-29T16:54:30.835288", "thread_id": null, "calling_id": "20241129-70470-0006-2879", "agent_name": "MyRun.4554695216", "model_name": null}'}
SSE >> {'id': '20241129-70470-0071-1100', 'event': 'message', 'data': '{"block_id": "20241129-70470-0071-1100", "block_type": "text", "content": "illufly!\\n", "content_id": "20241129-70470-0072-3321", "created_at": "2024-11-29T16:54:30.835666", "thread_id": null, "calling_id": "20241129-70470-0006-2879", "agent_name": "MyRun.4554695216", "model_name": null}'}


In [37]:
r.events_history.store

{'20241129-70470-0004-4087': {'agents': {'MyRun.4554695216': {}},
  'callings': {'20241129-70470-0004-7364': [{'id': '20241129-70470-0047-9401',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0047-9401", "block_type": "runnable", "content": "MyRun.4554695216", "content_id": "20241129-70470-0048-5561", "created_at": "2024-11-29T16:54:30.790858", "thread_id": null, "calling_id": "20241129-70470-0004-7364", "agent_name": "MyRun.4554695216", "model_name": null}'},
    {'id': '20241129-70470-0049-6304',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0049-6304", "block_type": "text", "content": "hi\\n", "content_id": "20241129-70470-0050-2090", "created_at": "2024-11-29T16:54:30.790924", "thread_id": null, "calling_id": "20241129-70470-0004-7364", "agent_name": "MyRun.4554695216", "model_name": null}'},
    {'id': '20241129-70470-0051-0908',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0051-0908", "block_type": "text", "content

In [38]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

['hi\n', 'illufly!\n']

### Runnable 事件流

In [39]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

def handle(block, **kwargs):
    print(block.json)

r = MyRun()
r("hi")

hi, illufly!

In [40]:
r.events_history.store

{'20241129-70470-0006-1285': {'agents': {'MyRun.4552445760': {}},
  'callings': {'20241129-70470-0008-2006': [{'id': '20241129-70470-0077-4352',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0077-4352", "block_type": "runnable", "content": "MyRun.4552445760", "content_id": "20241129-70470-0078-0523", "created_at": "2024-11-29T16:54:30.864311", "thread_id": null, "calling_id": "20241129-70470-0008-2006", "agent_name": "MyRun.4552445760", "model_name": null}'},
    {'id': '20241129-70470-0079-9418',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0079-9418", "block_type": "chunk", "content": "hi, ", "content_id": "20241129-70470-0080-5413", "created_at": "2024-11-29T16:54:30.864421", "thread_id": null, "calling_id": "20241129-70470-0008-2006", "agent_name": null, "model_name": null}'},
    {'id': '20241129-70470-0081-9932',
     'event': 'message',
     'data': '{"block_id": "20241129-70470-0081-9932", "block_type": "chunk", "content": "illufly",

### ChatAgent 事件流

In [41]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

def my_handler(block, **kwargs):
    print("[", block.block_type, "]", block.text)

r = FakeLLM(handlers=[my_handler], sleep=0.5)
r("hi")

[ runnable ] FakeLLM.4554697856
[ human ] hi
[ agent ] FakeLLM.4554697856
[ info ] 记住 10 轮对话
[ info ] I am FakeLLM
[ chunk ] R
[ chunk ] e
[ chunk ] p
[ chunk ] l
[ chunk ] y
[ chunk ]  
[ chunk ] >
[ chunk ] >
[ chunk ]  
[ chunk ] h
[ chunk ] i
[ new_line ] 
[ final_text ] Reply >> hi


'Reply >> hi'

In [42]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

r = FakeLLM(handlers=[alog], sleep=0.1)
await r("hi")

[HUMAN] hi
[AGENT] FakeLLM.4550277968
Reply >> hi


'Reply >> hi'

In [43]:
r.events_history.store

{'20241129-70476-0008-6446': {'agents': {'FakeLLM.4550277968': {'thread_id': '20241129-70476-0002-8351'}},
  'callings': {'20241129-70476-0010-6537': [{'id': '20241129-70476-0120-0402',
     'event': 'message',
     'data': '{"block_id": "20241129-70476-0120-0402", "block_type": "runnable", "content": "FakeLLM.4550277968", "content_id": "20241129-70476-0121-1364", "created_at": "2024-11-29T16:54:36.437922", "thread_id": null, "calling_id": "20241129-70476-0010-6537", "agent_name": "FakeLLM.4550277968", "model_name": null}'},
    {'id': '20241129-70476-0122-3820',
     'event': 'message',
     'data': '{"block_id": "20241129-70476-0122-3820", "block_type": "human", "content": "hi", "content_id": "20241129-70476-0123-0453", "created_at": "2024-11-29T16:54:36.440295", "thread_id": null, "calling_id": "20241129-70476-0010-6537", "agent_name": "FakeLLM.4550277968", "model_name": null}'},
    {'id': '20241129-70476-0124-8656',
     'event': 'message',
     'data': '{"block_id": "20241129-704

### 推送 ChatAgent 事件流

In [44]:
from illufly.chat import FakeLLM

llm = FakeLLM(name="if1", handlers=[])
llm.create_new_thread()

async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'id': '20241129-70477-0159-6847', 'event': 'message', 'data': '{"block_id": "20241129-70477-0159-6847", "block_type": "agent", "content": "if1", "content_id": "20241129-70477-0160-1141", "created_at": "2024-11-29T16:54:37.629609", "thread_id": "20241129-70477-0003-9737", "calling_id": "20241129-70477-0011-1841", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-70477-0161-7806', 'event': 'message', 'data': '{"block_id": "20241129-70477-0161-7806", "block_type": "info", "content": "记住 10 轮对话", "content_id": "20241129-70477-0162-4151", "created_at": "2024-11-29T16:54:37.630746", "thread_id": "20241129-70477-0003-9737", "calling_id": "20241129-70477-0011-1841", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-70477-0163-4935', 'event': 'message', 'data': '{"block_id": "20241129-70477-0163-4935", "block_type": "info", "content": "I am FakeLLM", "content_id": "901d4fdeae2f11ef949cacde48001122", "created_at": "2024-11-29T16:54:37.631805", "thread_id": "20241129-70477-00

In [45]:
async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'id': '20241129-70479-0214-4871', 'event': 'message', 'data': '{"block_id": "20241129-70479-0214-4871", "block_type": "agent", "content": "if1", "content_id": "20241129-70479-0215-2964", "created_at": "2024-11-29T16:54:39.833315", "thread_id": "20241129-70477-0003-9737", "calling_id": "20241129-70479-0012-2365", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-70479-0216-6090', 'event': 'message', 'data': '{"block_id": "20241129-70479-0216-6090", "block_type": "info", "content": "记住 10 轮对话", "content_id": "20241129-70479-0217-8095", "created_at": "2024-11-29T16:54:39.834410", "thread_id": "20241129-70477-0003-9737", "calling_id": "20241129-70479-0012-2365", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-70479-0218-4417', 'event': 'message', 'data': '{"block_id": "20241129-70479-0218-4417", "block_type": "info", "content": "I am FakeLLM", "content_id": "916da6d6ae2f11ef949cacde48001122", "created_at": "2024-11-29T16:54:39.835954", "thread_id": "20241129-70477-00

In [46]:
llm.events_history.store

{'20241129-70477-0009-4533': {'agents': {'if1': {'thread_id': '20241129-70477-0003-9737'}},
  'callings': {'20241129-70477-0011-1841': [{'id': '20241129-70477-0155-3377',
     'event': 'message',
     'data': '{"block_id": "20241129-70477-0155-3377", "block_type": "runnable", "content": "if1", "content_id": "20241129-70477-0156-2171", "created_at": "2024-11-29T16:54:37.629394", "thread_id": "20241129-70477-0003-9737", "calling_id": "20241129-70477-0011-1841", "agent_name": "if1", "model_name": null}'},
    {'id': '20241129-70477-0157-7784',
     'event': 'message',
     'data': '{"block_id": "20241129-70477-0157-7784", "block_type": "human", "content": "给我写一篇200字小作文", "content_id": "20241129-70477-0158-5354", "created_at": "2024-11-29T16:54:37.629551", "thread_id": "20241129-70477-0003-9737", "calling_id": "20241129-70477-0011-1841", "agent_name": "if1", "model_name": null}'},
    {'id': '20241129-70477-0159-6847',
     'event': 'message',
     'data': '{"block_id": "20241129-70477-015

### 嵌套 BaseAgent

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReAct(
    ChatQwen(name="react-planner", tools=[get_city, get_weather, booking])
)

for b in flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False, generator="sync"):
    print(b)

{'id': '20241129-70759-0002-7279', 'event': 'message', 'data': '{"block_id": "20241129-70759-0002-7279", "block_type": "node", "content": ">>> flow-node-1: planner", "content_id": "20241129-70759-0003-6557", "created_at": "2024-11-29T16:59:19.897398", "thread_id": null, "calling_id": "20241129-70759-0000-1712", "agent_name": "ReAct.4586590496", "model_name": null}'}
{'id': '20241129-70759-0006-3336', 'event': 'message', 'data': '{"block_id": "20241129-70759-0006-3336", "block_type": "agent", "content": "react-planner", "content_id": "20241129-70759-0007-0672", "created_at": "2024-11-29T16:59:19.897846", "thread_id": null, "calling_id": "20241129-70759-0000-1712", "agent_name": "react-planner", "model_name": "qwen-plus"}'}
{'id': '20241129-70759-0008-5017', 'event': 'message', 'data': '{"block_id": "20241129-70759-0008-5017", "block_type": "info", "content": "记住 10 轮对话", "content_id": "20241129-70759-0009-3579", "created_at": "2024-11-29T16:59:19.897933", "thread_id": null, "calling_id"

In [2]:
flow.events_history.store

{'20241129-70759-0006-5746': {'agents': {'ReAct.4586590496': {},
   'react-planner': {'thread_id': '20241129-70759-0000-7334'},
   'get_city': {}},
  'callings': {'20241129-70759-0000-1712': [{'id': '20241129-70759-0000-7036',
     'event': 'message',
     'data': '{"block_id": "20241129-70759-0000-7036", "block_type": "runnable", "content": "ReAct.4586590496", "content_id": "20241129-70759-0001-0428", "created_at": "2024-11-29T16:59:19.897083", "thread_id": null, "calling_id": "20241129-70759-0000-1712", "agent_name": "ReAct.4586590496", "model_name": null}'},
    {'id': '20241129-70759-0002-7279',
     'event': 'message',
     'data': '{"block_id": "20241129-70759-0002-7279", "block_type": "node", "content": ">>> flow-node-1: planner", "content_id": "20241129-70759-0003-6557", "created_at": "2024-11-29T16:59:19.897398", "thread_id": null, "calling_id": "20241129-70759-0000-1712", "agent_name": "ReAct.4586590496", "model_name": null}'},
    {'id': '20241129-70759-0004-7467',
     'eve

## Events History

### 默认历史事件文件夹

In [3]:
from illufly.io import LocalFileEventsHistory
from illufly.chat import FakeLLM
llm = FakeLLM(events_history=LocalFileEventsHistory())
llm.events_history.directory

'/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/__ILLUFLY__/LOCAL_FILE_EVENTS'

### 修改历史事件文件夹

In [4]:
from illufly.io import LocalFileEventsHistory
from illufly.chat import FakeLLM
llm = FakeLLM(events_history=LocalFileEventsHistory("__EVENTS__"))
llm.events_history.directory

'__EVENTS__'

In [5]:
# 默认加载最近的历史清单
print("当前历史：", llm.events_history.events_history_id)
llm.events_history.store[llm.events_history.events_history_id]

当前历史： 20241129-70910-0009-0421


{'agents': {}, 'callings': {}}

In [7]:
llm("hi")

[HUMAN] hi
[AGENT] FakeLLM.4440936976
Reply >> hi


'Reply >> hi'

### 列举历史清单

In [8]:
llm.events_history.list_events_histories()

['20241129-70910-0009-0421']

### 创建新的历史清单

In [9]:
llm.events_history.create_new_history()

'20241129-70984-0010-3256'

In [10]:
llm("我是新的历史")

[HUMAN] 我是新的历史
[AGENT] FakeLLM.4440936976
Reply >> 我是新的历史


'Reply >> 我是新的历史'

In [11]:
llm.events_history.list_events_histories()

['20241129-70910-0009-0421', '20241129-70984-0010-3256']

### 加载指定历史清单

In [12]:
llm.events_history.load_events_history('20241129-70910-0009-0421')
print("当前历史：", llm.events_history.events_history_id)

当前历史： 20241129-70910-0009-0421


{'agents': {'FakeLLM.4440936976': {'thread_id': '20241129-70945-0001-8756'}},
 'callings': {'20241129-70945-0001-2680': [{'id': '20241129-70945-0686-1524',
    'event': 'message',
    'data': '{"block_id": "20241129-70945-0686-1524", "block_type": "runnable", "content": "FakeLLM.4440936976", "content_id": "20241129-70945-0687-1811", "created_at": "2024-11-29T17:02:25.498553", "thread_id": null, "calling_id": "20241129-70945-0001-2680", "agent_name": "FakeLLM.4440936976", "model_name": null}'},
   {'id': '20241129-70945-0688-9608',
    'event': 'message',
    'data': '{"block_id": "20241129-70945-0688-9608", "block_type": "human", "content": "hi", "content_id": "20241129-70945-0689-2778", "created_at": "2024-11-29T17:02:25.498715", "thread_id": null, "calling_id": "20241129-70945-0001-2680", "agent_name": "FakeLLM.4440936976", "model_name": null}'},
   {'id': '20241129-70945-0690-9369',
    'event': 'message',
    'data': '{"block_id": "20241129-70945-0690-9369", "block_type": "agent", 

In [13]:
llm("新请求")

[HUMAN] 新请求
[AGENT] FakeLLM.4440936976
Reply >> 新请求


'Reply >> 新请求'

In [14]:
llm.events_history.store[llm.events_history.events_history_id]

{'agents': {'FakeLLM.4440936976': {'thread_id': '20241129-70945-0001-8756'}},
 'callings': {'20241129-70945-0001-2680': [{'id': '20241129-70945-0686-1524',
    'event': 'message',
    'data': '{"block_id": "20241129-70945-0686-1524", "block_type": "runnable", "content": "FakeLLM.4440936976", "content_id": "20241129-70945-0687-1811", "created_at": "2024-11-29T17:02:25.498553", "thread_id": null, "calling_id": "20241129-70945-0001-2680", "agent_name": "FakeLLM.4440936976", "model_name": null}'},
   {'id': '20241129-70945-0688-9608',
    'event': 'message',
    'data': '{"block_id": "20241129-70945-0688-9608", "block_type": "human", "content": "hi", "content_id": "20241129-70945-0689-2778", "created_at": "2024-11-29T17:02:25.498715", "thread_id": null, "calling_id": "20241129-70945-0001-2680", "agent_name": "FakeLLM.4440936976", "model_name": null}'},
   {'id': '20241129-70945-0690-9369',
    'event': 'message',
    'data': '{"block_id": "20241129-70945-0690-9369", "block_type": "agent", 